<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Introduction to Simulation with Python (Part 2)

Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)


<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

### Building a Simulation Model with Simpy, including:

* ### Shared resources
* ### Entities
* ### Entities Arrival / Generator

Credits: The following content is adapted from the official [carwash example](https://simpy.readthedocs.io/en/latest/examples/carwash.html) 

## Car wash example
- The Carwash example is a simulation of a carwash with a limited number of machines and a number of cars that arrive at the carwash to get cleaned.







- The carwash uses a Resource to model the limited number of washing machines. It also defines a process for washing a car.

- When a car arrives at the carwash, it requests a machine. Once it got one, it starts the carwash’s wash processes and waits for it to finish. It finally releases the machine and leaves.

- The cars are generated by another process. After creating an initial amount of cars it creates new car processes after a random time interval as long as the simulation continues.

Now we have understood the basics, we're going to give more structure. We will use the following convention:

- Resources will be modeled by a python class
- Entities will be modeled by a python class
- Entities arrivals will be modeled by a python function (*simpy process*)

This convention is not mandatory for working with Simpy, but it helps to better understand the process flow and makes the code more readable. We will review every component in the following.

In [2]:
import itertools
import random
import simpy

## Shared Resources

Shared resources are a way to model Process Interaction. They form a congestion point where processes queue up in order to use them. Resources can be used by a limited number of processes at a time (e.g. a gas station can be modelled as a resource with a limited amount of fuel-pumps). Processes request these resources to become a user (or to “own” them) and have to release them once they are 
done.

We create the simpy resource and its capacity using the `simpy.Resource` within the `__init__` method in the class. We then define every process this resource supports as a different method. 

In the CarWash example, washing the car is the process supported by the `CarWash` resource. Therefore, we define a `wash` method within the resource class

In [3]:
class Carwash:
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).
    """

    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        yield self.env.timeout(self.washtime)
        pct_dirt = random.randint(50, 99)
        print(f"Carwash removed {pct_dirt}% of {car}'s dirt.")

**Note the difference between the resource (`CarWash`) and the simpy resource (`self.machine = simpy.Resource()`)**

## Entities

Entities are the **things that move through the system** (e.g., products, documents, customers, phone calls, orders, raw materials, etc.)

They **arrive** to the system and **flow through it** (in most cases, using the **resources** of the system). We model the "flow" of the entity in the system as a class method. 

In the CarWash example, once a car have arrived to the system, it has to wait for a machine to become available afther which the car is washed (by the machine) and leaves the system. We define this "flow" in the `run` method within the entity class.

In [4]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name

    def run(self, carwash):
        print(f'{self.name} arrives at the carwash at {self.env.now:.2f}.')
        with carwash.machine.request() as request:
            yield request

            print(f'{self.name} enters the carwash at {self.env.now:.2f}.')
            yield self.env.process(carwash.wash(self.name))

            print(f'{self.name} leaves the carwash at {self.env.now:.2f}.')

The `run` method uses the `carwash` resource (and that's why is given as a parameter). 

For accessing a (simpy) resource we use the `.request()` statement. We wait for access to the resource using the `yield req` statement and finally we process (wash) the entity (car)

## Entity Generator

The **arrival / generation of entitites** is model as a process. **Note that we've separated the flow of an entity from the arrival of it**. These are two different (simpy) processes.  

In the CarWash example, a process is responsable of generating the cars. We make the entitites flow through the system (`env.process(car.run(carwash))`) within this process.

In [5]:
def car_generator(env, t_inter, carwash):
    car_count = itertools.count()

    # Create 4 initial cars
    for _ in range(4):
        car = Car(env, name=f'Car {next(car_count)}')
        env.process(car.run(carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        car = Car(env, name=f'Car {next(car_count)}')
        env.process(car.run(carwash))

## Run Simulation

Finally, we run the simulation. In general these are the steps:

1. Define "global" parameters
2. Create the simpy environment (`simpy.Environment()`)
3. Define the resources
4. Define the processes
5. Execute the simulation (`env.run`)

For steps 3, 4 and 5 we have to use the environment created in step 2

Note that the flow of the entity (`car.run` process) is called within the `car_generator` process. Therefore, we only have to call the `car_generator` "outside".

In [6]:
# parameters
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME =  5     # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~T_INTER minutes
SIM_TIME = 20     # Simulation time in minutes


# Setup and start the simulation
print('Running Simulation...')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.Environment()

#define resources
carwash = Carwash(env, NUM_MACHINES, WASHTIME)

#define processes
env.process(car_generator(env, T_INTER, carwash))

# Execute
env.run(until=SIM_TIME)
print('... Done')

Running Simulation...
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 5.00.
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 5 arrives at the carwash at 10.00.
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Carwash removed 97% of Car 4's dirt.
Carwash removed 56% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leaves the carwash at 15.00.
Car 6 arrives at the carwash at 16.00.
Car 6 enters the carwash at 16.00.
... Done


<img src="images/d3.png" style="width:50%; float:center;" />
